<a href="https://colab.research.google.com/github/Paolo20224/Prueba/blob/main/Simulacion_Examen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
#Como parte inicial, descargamos la informacion y la cargamos en nuestro cuader en Colab
import pandas as pd
filename = "/content/literacy_rates.csv"
df = pd.read_csv(filename)

In [54]:
# A continuacion revisaremos aquellos datos nulos para ver la integridad de la informacion
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4955 entries, 0 to 4954
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Region         4950 non-null   object
 1   Country        4955 non-null   object
 2   Year           4955 non-null   int64 
 3   Age            4955 non-null   object
 4   Gender         4955 non-null   object
 5   Literacy rate  4955 non-null   object
dtypes: int64(1), object(5)
memory usage: 232.4+ KB


In [55]:
# Revisaremos los primero diez datos para dimensionar la magnitud de la info
df.head(10)

,Region,Country,Year,Age,Gender,Literacy rate
0,Central and Southern Asia,Afghanistan,2011,15+,female,0.1761206
1,Central and Southern Asia,Afghanistan,2011,15+,male,0.454171
2,Central and Southern Asia,Afghanistan,2011,15+,total,0.3174112
3,NaN,Afghanistan,2011,15-24,female,0.3211322
4,NaN,Afghanistan,2011,15-24,male,0.6187907
5,Central and Southern Asia,Afghanistan,2011,15-24,total,0.46990049999999994
6,Central and Southern Asia,Afghanistan,2011,25-64,female,0.08412760000000001
7,Central and Southern Asia,Afghanistan,2011,25-64,male,0.3779481
8,Central and Southern Asia,Afghanistan,2011,25-64,total,0.2315362
9,Central and Southern Asia,Afghanistan,2011,65+,female,0.19670490000000002


In [56]:
#A continuacion revisaremos el archivo en excel para validar que elementos conducentes a error puede contener esta base.
df.to_excel("df_literacy_rates.xlsx", index=False)

In [57]:
#Lo primero que logramos visualizar son elementos nulos para las regiones para los paises de Afghanistan y Uzbekistan (cuya region es unica por pais), por lo que rellenaremos esta info en nuestro Dataframe
df2=df
df2["Region"].fillna("Central and Southern Asia", inplace=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4955 entries, 0 to 4954
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Region         4955 non-null   object
 1   Country        4955 non-null   object
 2   Year           4955 non-null   int64 
 3   Age            4955 non-null   object
 4   Gender         4955 non-null   object
 5   Literacy rate  4955 non-null   object
dtypes: int64(1), object(5)
memory usage: 232.4+ KB


In [60]:
#Otro de los elementos que se logra visualizar como una potencial contaminacion de informacion, es el hecho que la columna "Literacy Rate" tenga la clasificacion de Objeto, siendo que es un ratio y se espera una clasificacion tipo float
df2['Literacy rate'] = df2['Literacy rate'].replace("%", "")
df2['Literacy rate'] = df2['Literacy rate'].replace(".", ",").astype(float)

ValueError: ignored

In [ ]:
#Habiendo realizado el cambio anterior, podemos validar que "Literacy Rate" paso de object a float64
df2.info()

In [ ]:
# A continuacion logramos visualizar que existe un valor mayor a 1, considerando que dichos valores son ratios, este es un dato que no podemos aceptar.
df2.describe()

In [ ]:
df3=df
literacy_filter=df3["Literacy rate"]>1
df3=df3.loc[literacy_filter,:]
df3

In [ ]:
df2["Literacy rate"].replace([45.384],0.45384)
df2.describe()